In [ ]:
from pathlib import Path
import os

def bundle_sources(root=None, out="app_bundle.md", max_bytes=2_000_000):
    """
    Walks the codebase and writes a single Markdown file with headers + code fences
    so you can paste everything into ChatGPT.

    - If root is None, it auto-detects: uses 'app' if present, else current dir.
    - Skips typical build/cache dirs and obvious binaries.
    - Skips the output file itself and .ipynb notebooks.
    """
    # --- Auto-detect root ---
    if root is None:
        root = "app" if Path("app").is_dir() else "."
    root = Path(root).resolve()
    out = Path(out).resolve()

    # --- What to include / exclude ---
    INCLUDE_EXTS = {
        ".py", ".js", ".ts", ".tsx", ".jsx",
        ".html", ".css", ".scss", ".sass",
        ".json", ".md", ".sql", ".txt",
        ".yml", ".yaml", ".toml", ".ini",
        ".sh", ".ps1", ".bat",
    }
    EXCLUDE_DIRS = {
        ".git", ".svn", ".hg", ".DS_Store",
        "node_modules", "dist", "build", ".next", ".nuxt", "out",
        ".venv", "venv", "__pycache__", ".pytest_cache",
        ".idea", ".vscode", ".cache", ".mypy_cache",
        "coverage", "logs", "tmp", "staticfiles"
    }
    EXCLUDE_FILES = {
        out.name,                         # don't re-ingest the bundle we write
    }
    EXCLUDE_SUFFIXES = {
        ".ipynb",                         # skip notebooks
    }

    def lang_for(path: Path) -> str:
        if path.name == "Dockerfile":
            return "dockerfile"
        m = {
            ".py":"python", ".js":"javascript",
            ".ts":"ts", ".tsx":"tsx", ".jsx":"jsx",
            ".html":"html", ".css":"css", ".scss":"scss", ".sass":"sass",
            ".sql":"sql", ".json":"json", ".md":"md", ".txt":"txt",
            ".yml":"yaml", ".yaml":"yaml", ".toml":"toml", ".ini":"ini",
            ".sh":"bash", ".ps1":"powershell", ".bat":"bat",
        }
        return m.get(path.suffix.lower(), "")

    # --- Collect files ---
    files = []
    for dirpath, dirnames, filenames in os.walk(root):
        # prune excluded dirs
        dirnames[:] = [d for d in dirnames if d not in EXCLUDE_DIRS]
        for fn in filenames:
            p = Path(dirpath) / fn
            try:
                # file-level skips
                if p.name in EXCLUDE_FILES:
                    continue
                if p.suffix.lower() in EXCLUDE_SUFFIXES:
                    continue

                # include rules
                # include by extension, or Dockerfile specifically
                if p.name == "Dockerfile":
                    pass
                elif p.suffix.lower() not in INCLUDE_EXTS:
                    continue

                # size guard
                if p.stat().st_size > max_bytes:
                    continue

                files.append(p)
            except Exception:
                # ignore unreadable files
                continue

    files.sort(key=lambda p: str(p))

    # --- Write bundle ---
    out.parent.mkdir(parents=True, exist_ok=True)
    with out.open("w", encoding="utf-8", newline="\n") as f:
        f.write(f"# Source Bundle for `{root.name}`\n\n")
        f.write("> Generated in Jupyter. Each section shows the file path and its contents.\n\n")
        for p in files:
            # paths we walk are inside root, so this is safe
            rel = p.relative_to(root)
            lang = lang_for(p)
            try:
                text = p.read_text(encoding="utf-8", errors="replace")
            except Exception:
                continue
            f.write(f"---\n\n## `{rel.as_posix()}`\n\n```{lang}\n{text}\n```\n\n")

    print(f"Scanning: {root}")
    print(f"Wrote {len(files)} files → {out}")

# === Run it ===
# Auto-detects 'app' vs current folder, writes app_bundle.md next to the notebook.
bundle_sources(root=None, out="app_bundle.md")


Wrote 0 files → C:\Users\Austin\OneDrive\Documents\Admin\Business_Plan\Gutterz\01_Marketing\01_Website\duckworks\app\app_bundle.md
